<a href="https://colab.research.google.com/github/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Chapter_2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2 - Classification

For a classification problem, we want to build a model that can learn how different features (properties) of a system contribute **qualitatively** to that system, i.e. which features are characteristic for a certain class of objects.

The typical architecture of such an ANN is:

| Layer      | Description |
| ---------- | ----------- |
| Input      |Takes numeric coded features as input|
| Hidden     |Optional dense layers|
| Output     |Non-linear output to predict probabilities|
|Loss function|Cross entropy|


<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Classification%20network%20topology.jpg?raw=true" width="740" height="460" />

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns

# Set figure style
sns.set(style="ticks")

# Binary classification

In the first section, we will build a model that can predict whether a cell belongs to Mycobacteria or Corynebacteria based on the measured cell size and cell density.

We will cover the following basic concepts:
- How can we transfer our ANN constructed for a regression problem to a binary classification problem?
- How can we use the validation holdout method to monitor how our model performs on unseen data?


**First, we create a toy data set representing populations of 2 different bacteria (500 cells each)**.

The cell size of each cell is plotted against the cell density.

Labels:
- 0 - Mycobacterium
- 1 - Corynebacterium

In [ ]:
# Create toy data set
from sklearn.datasets import make_blobs

# Create clusters
X_total, y_total = make_blobs(n_samples=1000,
                              centers=2,
                              center_box=(10,20),
                              cluster_std=1,
                              random_state=42,
                              shuffle=True)

# Convert labels (0,1) to names
y_names = []
for i in y_total:
  if i == 0:
    y_names.append("Mycobacterium")
  else:
    y_names.append("Corynebacterium")

# Visualize data set
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, edgecolor="k")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

---
❓**Question**: How would a human solve the classification problem? How would a machine solve it?

---

**Now we draw a function that can separate the two cell clusters.**

In [ ]:
# Create a line to separate the clusters with slope m and intercept n
m = 1.15
n = 0
x_line = np.arange(10,20,1)
y_line = m*x_line + n

# Visualize data set
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, alpha=0.5, edgecolor="k")
plt.plot(x_line, y_line, color="green")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

Now we can measure **accuracy**. This metric tells us how often the predicted label is the same as the true label.

> $Accuracy = \frac {\text {Number correct predictions}} {\text {Number total predictions}}$

In [ ]:
# Define number of wrong predictions (y_pred_wrong)
# and number of total predictions (y_pred_total)
y_pred_wrong = 4
y_pred_total = 1000

y_pred_correct = y_pred_total - y_pred_wrong
accuracy = y_pred_correct / y_pred_total
print(f"Accuracy = {accuracy*100} %")

**Now we will split the data set into three parts: train (70%), test (20%), validation (10%).**
<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Holdout%20validation.jpg?raw=true" width="740" height="460" />

In [ ]:
# Split data set into train, test and validation set
X_train, X_test, y_train, y_test = train_test_split(X_total,
                                                    y_total,
                                                    train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  train_size=0.875)

# Check the split and put all values in dataframe
data = [[X_train.shape, len(X_train), 100 * len(X_train) / len(X_total)],
        [y_train.shape, len(y_train), 100 * len(y_train) / len(y_total)],
        [X_test.shape, len(X_test), 100 * len(X_test) / len(X_total)],
        [y_test.shape, len(y_test), 100 * len(y_test) / len(y_total)],
        [X_val.shape, len(X_val), 100 * len(X_val) / len(X_total)],
        [y_val.shape, len(y_val), 100 * len(y_val) / len(y_total)]]
index = ["X_train", "y_train","X_test", "y_test","X_val", "y_val",]
columns = ["Shape", "Elements", "Percentage"]
pd.DataFrame(data=data,
             columns=columns,
             index=index)

In [ ]:
# Visualize data sets
plt.scatter(X_train[:,0],X_train[:,1], label="Train", edgecolor="k")
plt.scatter(X_test[:,0],X_test[:,1], label="Test", edgecolor="k")
plt.scatter(X_val[:,0],X_val[:,1], label="Validation", edgecolor="k")
plt.legend()
plt.xlabel("Cell size")
plt.ylabel("Cell density");

## Binary Classification Model 1

---
Let's build our first neural network with the following components:
* **1 single dense neuron** (input and output are together)
* Sigmoid activation function **$f(x) = \frac {1}{1+e^{-x}}$**.
* Optimizer is **Stochastic Gradient Descent SGD**.
* Loss function to minimize: **Binary Crossentropy**
* Metrics to monitor during training: **Accuracy**
* Validation set **(X_val, y_val)**
* Train model for **5 epochs**

<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Binary%20classification%20model%201.jpg?raw=true" width="740" height="460" />

In [ ]:
# Set up random seed for reproducibility
tf.random.set_seed(42)

# Create model
model_1 = keras.Sequential([
    keras.Input(shape=(X_train.shape[1:])),
    layers.Dense(units=1, activation="sigmoid")
    ])

model_1.summary()

# Compile model
model_1.compile(optimizer="sgd",
                loss=keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

# Fit model
history_1 = model_1.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

**Now we will check the weights that our model has learned:**
- $w_1$, $w_2$ for the inputs $x_i$ (cell size) and $y_i$ (cell density)
- b for the bias neuron

Remember that these parameters define the calculation **before** the activation function.

In [ ]:
# Get weights and bias vector
w1, w2 = model_1.get_weights()[0]
b = model_1.get_weights()[1]
print(f"Weight 1 = {w1.item():.2f}")
print(f"Weight 2 = {w2.item():.2f}")
print(f"Bias neuron = {b.item():.2f}")

To understand how our model decides whether a cell is a mycobacterium or a corynebacterium, we will create artificial cells with all possible combinations of x (cell size) and y (cell density) and compute the model's computational steps separately for each of these cells.

**With the weights and the bias neuron, we can replicate the calculation done inside the neuron:** $\text{output before activation} = \displaystyle (\sum_{i=0}^n x_i * w_i) + b $ <br>
$ \iff \text{output before activation} = \displaystyle (x * w_1 + y * w_2) + b $

This will predict the output before the activation function.

In [ ]:
# Plot decision boundary with artificial cells

# Boundaries of plot
x_min = X_total[:, 0].min() - 1
x_max = X_total[:, 0].max() + 1
y_min = X_total[:, 1].min() - 1
y_max = X_total[:, 1].max() + 1

# Array with continous 40 points
x_array = np.linspace(x_min, x_max, 40)
y_array = np.linspace(y_min, y_max, 40)

# Combines every point from array 1 with every point of array 2
xx, yy = np.meshgrid(x_array, y_array)

# Create 2D-array with xx and yy as columns
points = np.column_stack((xx.ravel(), yy.ravel()))

# Calculates output with weights and bias neuron
mesh_output = (points[:,0]*w1 + points[:,1]*w2) + b

# Plot output before going to sigmoid function
sns.scatterplot(x=points[:,0], y=points[:,1], hue=mesh_output, hue_norm=(-2,2))
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1],hue=y_names, alpha=0.8, edgecolor="k")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.title("Output of the linear calculation before activation")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

Now we will transform the different output values by passing them to the **activation function (sigmoid)**.

Remember that the function looks like this:

In [ ]:
# Generate plot of sigmoid function
x_sigmoid = np.arange(-10,10,0.5)
y_sigmoid = tf.nn.sigmoid(x_sigmoid)
plt.plot(x_sigmoid, y_sigmoid)
plt.title("Sigmoid function")
plt.xlabel("Input")
plt.ylabel("Output");

❓**Question**: How do you expect the activation function to transform the artificial cell values?

In [ ]:
# Pass mesh output to sigmoid
final_output = tf.sigmoid(mesh_output)

# Plot output after sigmoid function
sns.scatterplot(x=points[:,0], y=points[:,1], hue=final_output)
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1],hue=y_names, alpha=0.8, edgecolor="k")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.title("Output of the neuron after activation by sigmoid")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

❓**Question**: Now that you have seen how the model arrives at its conclusions, is this the same method we used at the beginning to separate the two clusters?

**Let's compare the actual output of our model to the deconstructed model's output.**

In [ ]:
# Calculate and plot decision boundary of model_1
predicted_mesh = tf.squeeze(model_1.predict(points, verbose=0))
sns.scatterplot(x=points[:,0], y=points[:,1], hue=predicted_mesh)
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1],hue=y_names, alpha=0.8, edgecolor="k")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.title("Output of our model")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

**The common visualization technique for classification model decision boundaries is a contour plot. You can think of this as feeding a large number of artificial cells whose parameters change only slightly into the model and plotting its predictions**.

In [ ]:
# Reshape the mesh_output back to the grid shape
mesh_output = tf.reshape(predicted_mesh, shape=(xx.shape))

# Plot decision boundary as contour plot
plt.contourf(xx, yy, mesh_output, cmap='RdBu', alpha=0.8)
plt.colorbar()

# Plot data points
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, alpha=0.8, edgecolor="k")
plt.legend(bbox_to_anchor=(1.25, 1), loc='upper left', borderaxespad=0)
plt.title("Output of the model")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

It is important to note that by default, models predict probabilities **>0.5 as 1 and <0.5 as 0**. This means that there is a zone (**twilight zone**) in the decision boundary where the difference between different prediction outcomes is very small and highly influenced by the data (e.g., measurement error).

**In the next plot, the color scheme is adjusted to represent three ranges of values:**
- 0 - 0.45 : High confidence for label 0 (Mycobacterium)
- 0.45 - 0.55: Twilight zone
- 0.55 - 1.0: High confidence for label 1 (Corynebacterium)

In [ ]:
# Plot decision boundary as contour plot scaled to three bins
plt.contourf(xx, yy, mesh_output, cmap='RdBu', alpha=0.8, levels=[0, 0.45, 0.55, 1.0])
plt.colorbar()

# Plot data points
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, alpha=0.8, edgecolor="k")
plt.legend(bbox_to_anchor=(1.25, 1), loc='upper left', borderaxespad=0)
plt.title("Output of the model")
plt.xlabel("Cell size")
plt.ylabel("Cell density");

**Now that we've seen how our model arrives at its predictions, let's evaluate the model's accuracy on the different data set.**

In [ ]:
# Predict accuracy for test, validation and train data set
test_accuracy = model_1.evaluate(X_test, y_test, verbose=0)[1]
train_accuracy = model_1.evaluate(X_train, y_train, verbose=0)[1]
val_accuracy = model_1.evaluate(X_val, y_val, verbose=0)[1]
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Validation accuracy: {val_accuracy:.2f}")

**Now we find and visualize the wrong predictions and check their position on the contour plot.**

In [ ]:
# Find the wrong predictions
predictions_total = tf.round(model_1.predict(X_total, verbose=0))

# Create dataframe with values, y_true and y_pred
data_predictions = {"cell_size" : X_total[:, 0],
        "cell_density" : X_total[:, 1],
        "y_true" : y_total,
        "y_pred" : tf.squeeze(predictions_total)}
predictions_df = pd.DataFrame(data_predictions)

# Filter entries where y_true is not equal to y_pred
wrong_predictions = predictions_df[predictions_df["y_true"] != predictions_df["y_pred"]]

# Visualize data set
fig, ax = plt.subplots(1,2, constrained_layout = True, figsize=(10,4))

# Plot decision boundary as contour plot
contour = ax[1].contourf(xx, yy, mesh_output, cmap='RdBu', alpha=0.8)
plt.colorbar(contour, ax=ax[1])

# Plot scatterplots
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax[0], alpha=0.8, edgecolor="k", legend=None)
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax[1], alpha=0.8, edgecolor="k")
sns.scatterplot(data=wrong_predictions, x="cell_size", y="cell_density", color="red", ax=ax[1], edgecolor="k", label="Wrong predictions")
ax[0].set_xlabel("Cell size")
ax[1].set_xlabel("Cell size")
ax[1].legend(bbox_to_anchor=(-1., 1), loc='upper left', borderaxespad=0)
ax[0].set_ylabel("Cell density")
ax[1].set_ylabel(None);

❓**Question:** Take a close look at the wrong predictions. What do you think was the reason for the wrong prediction?

It is often useful to see how a model makes errors. For example, are the errors evenly distributed across the classes, or is one class in particular being mispredicted as another class?

Since in real-world applications an ANN will never be 100% accurate, it is often important to see how the errors are distributed. Very useful metrics for this are:

> Precision = $\frac {\text{true positives}} {\text{true positives + false positives}}$

> Recall = $\frac {\text{true positives}} {\text{true positives + false negatives}}$

Precision and recall behave antiproportionally over a wide range, and models can be tuned to maximize either metric. Precision is important for the question of whether a positive prediction is really positive. Recall is important for the question of whether all samples that are really positive have been predicted.

In [ ]:
# Calculate and plot confusion matrix
predictions_total = tf.round(model_1.predict(X_total, verbose=0))
ConfusionMatrixDisplay.from_predictions(y_true=y_total,
                                        y_pred=predictions_total,
                                        display_labels=["Mycobacterium", "Corynebacterium"]
                                        );

In [ ]:
# Calculate precision and recall score
from sklearn.metrics import precision_score, recall_score
model_1_precision = precision_score(y_true=y_total, y_pred=predictions_total)
model_1_recall = recall_score(y_true=y_total, y_pred=predictions_total)
print(f"model_1 precision score: {100 * model_1_precision:.1f}%")
print(f"model_1 recall score: {100 * model_1_recall:.1f}%")

Let's say we want to use the model in a detection method for *Mycobacterium tuberculosis*. Cells are sampled from patients, the cell size and cell density are automatically determined, and our model should judge whether the sampled cells could be a *Mycobacterium tuberculosis* cell.

❓**Question:** How would you design a model for this use case in terms of its precision and recall?

**We can change the the decision threshold from 0.5 to 0.65 to ensure that the model detects all mycobacterial cells.**

In [ ]:
# Calculate precision and recall score
predictions_total_65 = model_1.predict(X_total, verbose=0) >= 0.65
predictions_total_65 = predictions_total_65.astype(int)

# Plot confusion matrix with threshold 0.65
ConfusionMatrixDisplay.from_predictions(y_true=y_total,
                                        y_pred=predictions_total_65,
                                        display_labels=["Mycobacterium", "Corynebacterium"]
                                        );

We have seen how a model can predict whether a sample belongs to one class or another. Now we will use the concepts we learned to look at models that predict several different classes.

# Multi-class classification

In the second section, we will extend the data set to include cell densities and cell sizes measured for **Bacillus**, **Methanococcus**, and **Staphylococcus**. We will then build a multi-class model that predicts which of these 5 classes a cell belongs to.

We will cover the following basic concepts
- How can we transfer our ANN constructed for a binary classification problem to a multi-class classification problem?
- How can we visualize if our model is overfitting or not?
- What are the decision boundaries for a multi-class model?
- What is the black box problem of ANNs?

**First, we create a toy data set representing populations of 5 different bacteria (400 cells each)**.

The cell size of each cell is plotted against the cell density.

Labels:
- 0 - Mycobacterium
- 1 - Corynebacterium
- 2 - Bacillus
- 3 - Methanococcus
- 4 - Staphylococcus

In [ ]:
# Create clusters
X_total, y_total = make_blobs(n_samples=2000,
                              centers=5,
                              center_box=(5,20),
                              cluster_std=1,
                              random_state=42,
                              shuffle=True)

# Create a dictionary to map integer labels to organism names
organism_names_mapping = {
    0: "Mycobacterium",
    1: "Corynebacterium",
    2: "Bacillus",
    3: "Methanococcus",
    4: "Staphylococcus"
}

# Convert labels (0, 1, 2, 3, 4) to names
y_names = [organism_names_mapping[i] for i in y_total]

# Visualize data set
sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, edgecolor="k")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("Cell size")
plt.ylabel("Cell density");

**Now we split the whole data set into training, test and validation sets and visualize the split.**

In [ ]:
# Split data set into train, test and validation set
X_train, X_test, y_train, y_test = train_test_split(X_total,y_total, train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, train_size=0.875)

# Check the split and put all values in datafram
data = [[X_train.shape, len(X_train), 100 * len(X_train) / len(X_total)],
        [y_train.shape, len(y_train), 100 * len(y_train) / len(y_total)],
        [X_test.shape, len(X_test), 100 * len(X_test) / len(X_total)],
        [y_test.shape, len(y_test), 100 * len(y_test) / len(y_total)],
        [X_val.shape, len(X_val), 100 * len(X_val) / len(X_total)],
        [y_val.shape, len(y_val), 100 * len(y_val) / len(y_total)]]
index = ["X_train", "y_train","X_test", "y_test","X_val", "y_val",]
columns = ["Shape", "Elements", "Percentage"]
pd.DataFrame(data=data,
             columns=columns,
             index=index)

In [ ]:
# Visualize data sets
plt.scatter(X_train[:,0],X_train[:,1], label="Train", edgecolor="k")
plt.scatter(X_test[:,0],X_test[:,1], label="Test", edgecolor="k")
plt.scatter(X_val[:,0],X_val[:,1], label="Val", edgecolor="k")
plt.legend()
plt.xlabel("Cell size")
plt.ylabel("Cell density");

## Multi-class model 1

---
Let's build our first neural network for a multi-class classification with the following components:

* **5 dense neurons** (input and output are together).
* Activation function: **Softmax** **$f(x) = \frac {e^{x_j}}{\sum ^{n} _{i=1} e^{x_i}}$**.
* Optimizer is **Stochastic Gradient Descent SGD**.
* Loss function to minimize: **Sparse Categorical Crossentropy**.
* Metrics to monitor during training: **accuracy**.
* Validation set: **(X_val, y_val)**.
* Training model for **5 epochs**.

<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Multiclass%20classification%20model%201.jpg?raw=true" width="740" height="460" />

In [ ]:
# Set up random seed for reproducibility
tf.random.set_seed(42)

# Create model
model_2 = keras.Sequential([
    keras.Input(shape=(X_train.shape[1:])),
    layers.Dense(units=5,activation="softmax")])

model_2.summary()

# Compile model
model_2.compile(optimizer="sgd",
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=["accuracy"])

# Fit model
history_2 = model_2.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

**Plotting the loss functions and metrics used (in our case, accuracy) against trained epochs is a very useful tool to see if the model is over- or under-fitting.**

In [ ]:
# Plot accuracies and losses over epochs
pd.DataFrame(history_2.history).plot()
plt.xlabel("Epochs");

❓**Question**: What does the plot tell you?

**Let's examine the model's decision boundaries.**

In [ ]:
# Predict decision boundary

# Boundaries of plot
x_min = X_total[:, 0].min() - 1
x_max = X_total[:, 0].max() + 1
y_min = X_total[:, 1].min() - 1
y_max = X_total[:, 1].max() + 1

# Array with continous 100 points
x_array = np.linspace(x_min, x_max, 100)
y_array = np.linspace(y_min, y_max, 100)

# Combines every point from array 1 with every point of array 2
xx, yy = np.meshgrid(x_array, y_array)

# Create 2D-array with xx and yy as columns
points = np.column_stack((xx.ravel(), yy.ravel()))

# Predict output using meshgrid as input
mesh_output = model_2.predict(points, verbose=0)

# Split the output into the 5 probability channels
probability_channels = [tf.reshape(mesh_output[:, i], xx.shape) for i in range(5)]

# Output titles for plots
output_titles = ["Output 1", "Output 2", "Output 3", "Output 4", "Output 5"]

# Visualize data set
fig, axes = plt.subplots(1,5, constrained_layout = True, figsize=(20,4))

# Plot decision boundary and scatterplots
for i, ax in enumerate(axes):
    # Plot decision boundary as contour plot
    cnt = ax.contourf(xx, yy, probability_channels[i], cmap='RdBu', alpha=0.8)
    ax.set_title(output_titles[i])
    # Plot scatterplot and only display legend for first subplot
    if i == 0:
      sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax, alpha=0.8, edgecolor="k")
      ax.legend(bbox_to_anchor=(-0.7, 1), loc='upper left', borderaxespad=0)
    else:
      sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax, alpha=0.8, edgecolor="k", legend=None)
    # Remove axes labels and ticks
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticks([])
    ax.set_yticks([])

# Add a single colorbar for all subplots
plt.colorbar(cnt, ax=axes);

❓**Question**: What do the decision boundaries and confusion matrix tell you about the model?

In [ ]:
# Plot confusion matrix
predictions_total = model_2.predict(X_total, verbose=0)
ConfusionMatrixDisplay.from_predictions(y_true=y_total,
                                        y_pred=predictions_total.argmax(axis=1),
                                        display_labels=organism_names_mapping.values(),
                                        xticks_rotation=60
                                        );

## Multi-class Classification Model 2

**Exercise**: Now it is your turn again. Try to beat the previous model by adding complexity.

Network architecture and adjustable hyperparameters:

|Layer name|Number of neurons|Activation function|
|----------|-----------------|-------------------|
|Layer 1|units_1|activation_1|
|Layer 2|units_2|activation_2|
|Layer 3|units_3|activation_3|
|Output layer|units_output|activation_output|

---

<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Multi-class%20Classification%20Model%202.jpg?raw=true" width="740" height="460" />

In [ ]:
# Fill out the hyperparameters

# units_i is the number of neurons in layer i. Typical values are 0-100
units_1 =
units_2 =
units_3 =
units_output =
# activation_i is the activation function for layer i.
activation_1 =""
activation_2 =""
activation_3 =""
activation_output ="softmax"
# epochs
epochs =

# DO NOT CHANGE ANY OF THE FOLLOWING

# Set up random seed for reproducibility
tf.random.set_seed(42)

# Build model
model_3 = keras.Sequential([layers.Dense(units=units_1, activation=activation_1, input_shape=X_train.shape[1:]),
                            layers.Dense(units=units_2, activation=activation_2),
                            layers.Dense(units=units_3, activation=activation_3),
                            layers.Dense(units=units_output, activation=activation_output)],
                            name="Model_3")

model_3.summary()

# Compile model
model_3.compile(optimizer="sgd",
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=["accuracy"])

# Fit model
history_3 = model_3.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

# Plot accuracies and losses over epochs
pd.DataFrame(history_3.history).plot()
plt.xlabel("Epochs");

**Check the decision boundaries and confusion matrix with the next two cells.**

In [ ]:
# Predict decision boundary

# Boundaries of plot
x_min = X_total[:, 0].min() - 1
x_max = X_total[:, 0].max() + 1
y_min = X_total[:, 1].min() - 1
y_max = X_total[:, 1].max() + 1

# Array with continous 100 points
x_array = np.linspace(x_min, x_max, 100)
y_array = np.linspace(y_min, y_max, 100)

# Combines every point from array 1 with every point of array 2
xx, yy = np.meshgrid(x_array, y_array)

# Create 2D-array with xx and yy as columns
points = np.column_stack((xx.ravel(), yy.ravel()))

# Predict output using meshgrid as input
mesh_output = model_3.predict(points, verbose=0)

# Split the output into the 5 probability channels
probability_channels = [tf.reshape(mesh_output[:, i], xx.shape) for i in range(5)]

# Output titles for plots
output_titles = ["Output 1", "Output 2", "Output 3", "Output 4", "Output 5"]

# Visualize data set
fig, axes = plt.subplots(1,5, constrained_layout = True, figsize=(20,4))

# Plot decision boundary and scatterplots
for i, ax in enumerate(axes):
    # Plot decision boundary as contour plot
    cnt = ax.contourf(xx, yy, probability_channels[i], cmap='RdBu', alpha=0.8)
    ax.set_title(output_titles[i])
    # Plot scatterplot and only display legend for first subplot
    if i == 0:
      sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax, alpha=0.8, edgecolor="k")
      ax.legend(bbox_to_anchor=(-0.7, 1), loc='upper left', borderaxespad=0)
    else:
      sns.scatterplot(x=X_total[:, 0], y=X_total[:, 1], hue=y_names, ax=ax, alpha=0.8, edgecolor="k", legend=None)
    # Remove axes labels and ticks
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticks([])
    ax.set_yticks([])

# Add a single colorbar for all subplots
plt.colorbar(cnt, ax=axes);

In [ ]:
# Plot confusion matrix
predictions_total = model_3.predict(X_total, verbose=0)
ConfusionMatrixDisplay.from_predictions(y_true=y_total,
                                        y_pred=predictions_total.argmax(axis=1),
                                        display_labels=organism_names_mapping.values(),
                                        xticks_rotation=60
                                        );

**When you are satisfied with your model, evaluate it on the test data set and compare it to model_2.**

In [ ]:
# Evaluate model_2 and model_3 on test data set
model_2_eval = model_2.evaluate(X_test, y_test, verbose=0)[1] * 100
model_3_eval = model_3.evaluate(X_test, y_test, verbose=0)[1] * 100
plt.bar(x=["model_2", "model_3"], height=[model_2_eval, model_3_eval], edgecolor="k")
plt.title("Accuracy on test data set")
plt.ylabel("Accuracy[%]")
plt.ylim(0,105)

# Annotate each bar with its accuracy value
for i, v in enumerate([model_2_eval, model_3_eval]):
    plt.text(i, v + 1, f"{v:.2f}%", ha='center', va='bottom')

##Blackbox problem

**Let's review the weights of the first multi-class model and try to understand how the model arrives at its predictions.**

In [ ]:
# Extract weights and biases from model and put them in dataframe
model_2_weights_x, model_2_weights_y = model_2.get_weights()[0]
model_2_biases = model_2.get_weights()[1]
weights_biases_df = pd.DataFrame([model_2_weights_x, model_2_weights_y, model_2_biases],
                                 index=["x_weights", "y_weights", "bias_vectors"],
                                 columns=list(range(1,6))).transpose()

weights_biases_df

❓ **Question**: Can you explain how the model arrives at its decisions by looking at the weights and bias vectors?

The black box problem refers to the fact that in all but the simplest cases, humans are unable to understand the decision-making process of a deep learning model (this is also true for most classical machine learning models). Therefore, the model acts as a black box, receiving human-interpretable inputs and producing human-interpretable outputs, but the process between inputs and outputs is not human-interpretable.

# Image classification with the MNIST dataset

The third part of this chapter deals with one of the most complex classification problems: Image Classification or Computer Vision.

We will cover the following basic concepts:
- How can we transfer our ANN constructed for a multi-class classification problem to work on images?
- How is an image represented in the computer?
- Why are ANNs actually very rigid and not flexible with respect to the data they receive as input?

**We will use the MNIST data set, which consists of handwritten digits from 0 to 9.**

In [ ]:
# Import MNIST dataset
from keras.datasets import mnist

# The data has already been sorted into training and test sets for us
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [ ]:
# Check how the data is organized
print(f"train_data shape: {train_data.shape}")
print(f"train_labels shape: {train_labels.shape}")
print(f"test_data shape: {test_data.shape}")
print(f"test_labels shape: {test_labels.shape}")

The train_data consists of 60,000 images with a size of 28 x 28 (width x height).

**Let's see how an image is represented in numerical form.**

In [ ]:
# Check how one picture looks like
train_data[0]

In [ ]:
# What is the minimum and the maximum value?
print(f"Pixel values in range: {train_data[0].min()} - {train_data[0].max()}")

An image consists of an array of 28 x 28 pixels. The pixel values are integers (...,-2, -1, 0, 1, 2, ...) in the range 1-255. Since only one array is used for representation, the image is a grayscale image with values representing black (1) to white (255).

In [ ]:
# Plot first image and label
plt.imshow(train_data[0], cmap=plt.cm.gray)
plt.colorbar()
plt.xticks([])
plt.yticks([])
print(f"Label: {train_labels[0]}")

❓**Question**: How does a human know that this picture is a picture of a 5?

And how could a machine learn that?

## Model 1 for MNIST Classification

---
Let's build our first neural network for multi-class classification of images with the following components:

* **Flatten layer**: transforms 2D-image into 1D-array
* **10 dense neurons** (input and output are together).
* Activation function: **Softmax** **$f(x) = \frac {e^{x_j}}{\sum ^{n} _{i=1} e^{x_i}}$**.
* Optimizer is **Stochastic Gradient Descent SGD**.
* Loss function to minimize: **Sparse Categorical Crossentropy**.
* Metrics to monitor during training: **accuracy**.
* Validation set: **(X_val, y_val)**.
* Training model for **5 epochs**.

<img src="https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Image%20classification%201.jpg?raw=true" width="740" height="460" />

In [ ]:
# Set random set
tf.random.set_seed(42)

# Create model without normalization
model_3 = keras.Sequential([
    layers.Flatten(input_shape=train_data.shape[1:]),
    layers.Dense(10, activation="softmax")
    ])

model_3.summary()

# Compile model

model_3.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                       optimizer=keras.optimizers.SGD(),
                       metrics=["accuracy"])

# Train model

history_3 = model_3.fit(train_data, train_labels, epochs=5, batch_size=32)

Before we evaluate our first model, let's take a look at data normalization. We have seen that the images consist of pixel values between 1-255.

❓**Question**: Why might this be a problem for the neural network?

**Now we will normalize the data, which means that we will transform the pixel values from an interval of [1-255] to [0-1]. We can easily do this by dividing each pixel by the maximum pixel value in the image (255).**

In [ ]:
# Normalize data
print(f"Minimum pixel value: {train_data[0].min()} and maximum pixel value: {train_data[0].max()}")

train_data_normalized = train_data / train_data[0].max()
test_data_normalized = test_data / train_data[0].max()

print(train_data_normalized[0])

**Now we set up the exact same model as before, but use the normalized data set for training. Then we compare the performance of the model trained on the non-normalized data with this model.**

In [ ]:
# Set random set
tf.random.set_seed(42)

# Create model with normalization
model_3_normalized = keras.Sequential([
    layers.Flatten(input_shape=train_data.shape[1:]),
    layers.Dense(10, activation="softmax")
    ])

model_3_normalized.summary()

# Compile model

model_3_normalized.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                       optimizer=keras.optimizers.SGD(),
                       metrics=["accuracy"])

# Train model

history_3_normalized = model_3_normalized.fit(train_data_normalized, train_labels, epochs=5, batch_size=32)

In [ ]:
# Plot training of normalized and non-normalized model
fig, axes = plt.subplots(1,2, layout="constrained", figsize=(8,4))
axes[0].plot(history_3.history["loss"], label="Not normalized")
axes[1].plot(history_3.history["accuracy"], label="Not normalized")
axes[0].plot(history_3_normalized.history["loss"], label="Normalized")
axes[1].plot(history_3_normalized.history["accuracy"], label="Normalized")
axes[0].set_title("Loss")
axes[1].set_title("Accuracy")
axes[0].set_yscale("log")

for ax in axes:
  ax.set_xlabel("Epochs")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0);

**Let's examine the confusion matrix and try to understand where the model's predictions are good and where the model is struggling.**

In [ ]:
# Calculate and plot confusion matrix of best model
model_3_predictions = model_3_normalized.predict(train_data_normalized)
fig, axes = plt.subplots(figsize=(30,5), layout="constrained")
ConfusionMatrixDisplay.from_predictions(y_true=train_labels,
                                        y_pred=model_3_predictions.argmax(axis=1),
                                        ax=axes
                                        );

**Now we will visualize some of the incorrectly predicted images.**

In [ ]:
# Dataframe with true labels and predictions for train data
# Index is index in train_data
predictions_df =pd.DataFrame({"true_label" : train_labels,
                              "prediction" : model_3_predictions.argmax(axis=1)})

# Subdataframe with wrong predictions and sort
predictions_df_wrong = predictions_df[
    (predictions_df["true_label"] != predictions_df["prediction"])
    ].sort_values("true_label")

# Plot first image and label
fig, axes = plt.subplots(3,5, layout="constrained", figsize=(10,5))
# Fill first row with examples of true_label = 0
for i in range(5):
  index_list = predictions_df_wrong[predictions_df_wrong["true_label"] == 0].iloc[0:5].index
  axes[0,i].imshow(train_data_normalized[index_list[i]], cmap=plt.cm.gray)
  axes[0,i].set_title(str(predictions_df_wrong.loc[index_list[i]]).split("Name")[0])
  axes[0,i].set_xticks([])
  axes[0,i].set_yticks([])
# Fill second row with examples of true_label = 1
for i in range(5):
  index_list = predictions_df_wrong[predictions_df_wrong["true_label"] == 1].iloc[0:5].index
  axes[1,i].imshow(train_data_normalized[index_list[i]], cmap=plt.cm.gray)
  axes[1,i].set_title(str(predictions_df_wrong.loc[index_list[i]]).split("Name")[0])
  axes[1,i].set_xticks([])
  axes[1,i].set_yticks([])
# Fill 3 row with examples of true_label = 2
  index_list = predictions_df_wrong[predictions_df_wrong["true_label"] == 2].iloc[0:5].index
  axes[2,i].imshow(train_data_normalized[index_list[i]], cmap=plt.cm.gray)
  axes[2,i].set_title(str(predictions_df_wrong.loc[index_list[i]]).split("Name")[0])
  axes[2,i].set_xticks([])
  axes[2,i].set_yticks([])

## Model 2 for MNIST Classification



To understand the next core concept, we need a very good model. So we build a model with the following components:

* **Flatten layer**: transforms 2D-image into 1D-array
* **2 Hidden layers** with 16 dense neurons + ReLU
* Activation function: **Softmax** **$f(x) = \frac {e^{x_j}}{\sum ^{n} _{i=1} e^{x_i}}$**.
* Optimizer is **Adam**.
* Loss function to minimize: **Sparse Categorical Crossentropy**.
* Metrics to monitor during training: **accuracy**.
* Validation set: **(X_val, y_val)**.
* Training model for **10 epochs**.

In [ ]:
# Set random set
tf.random.set_seed(42)

# Create model without normalization
model_4 = keras.Sequential([
    layers.Flatten(input_shape=train_data.shape[1:]),
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(10, activation="softmax")
    ])

model_4.summary()

# Compile model
model_4.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                       optimizer=keras.optimizers.Adam(),
                       metrics=["accuracy"])

# Train model
history_4 = model_4.fit(train_data_normalized, train_labels, validation_split=0.1, epochs=20, batch_size=32)

In [ ]:
# Inspect training of model_4
pd.DataFrame(history_4.history).plot()

**Let's check the Confusion Matrix.**

In [ ]:
# Calculate and plot confusion matrix of model_4
model_4_predictions = model_4.predict(train_data_normalized)
fig, axes = plt.subplots(figsize=(30,5), layout="constrained")
ConfusionMatrixDisplay.from_predictions(y_true=train_labels,
                                        y_pred=model_4_predictions.argmax(axis=1),
                                        ax=axes
                                        );

So the model seems really good. **Now we will use an image of a handwritten 6 that I drew myself and see if our model can predict the number.** The image is the same size as the images in the train dataset (28 x 28) and the pixel values are normalized.

In [ ]:
# Load image, convert it to grayscale and scale it
!wget "https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Handwritten%206.jpg?raw=true"
image = plt.imread("/content/Handwritten 6.jpg?raw=true")
image = tf.image.rgb_to_grayscale(image)
image = image / 255
plt.imshow(image, cmap=plt.cm.gray)
plt.colorbar();

In [ ]:
# Predict class of image with model_4 and plot probabilities
prediction_image = model_4.predict(tf.expand_dims(image, axis=0), verbose=0)
fig, axes = plt.subplots(1,2, layout="constrained")
for i in range(2):
  axes[i].bar(height=prediction_image[0], x=list(range(10)))
  axes[i].set_ylabel("Probability")
  axes[i].set_xticks(list(range(10)))
  if i == 1:
    axes[i].set_yscale("log")
    axes[i].set_ylabel("")

The prediction is completely wrong.

❓**Question**: Do you have any idea what the reason might be?

In [ ]:
# Inverse image
image = 1 - image
plt.imshow(image, cmap=plt.cm.gray)
plt.colorbar();

In [ ]:
# Predict class of image with model_4 and plot probabilities
prediction_image = model_4.predict(tf.expand_dims(image, axis=0), verbose=0)
fig, axes = plt.subplots(1,2, layout="constrained")
for i in range(2):
  axes[i].bar(height=prediction_image[0], x=list(range(10)))
  axes[i].set_ylabel("Probability")
  axes[i].set_xticks(list(range(10)))
  if i == 1:
    axes[i].set_yscale("log")
    axes[i].set_ylabel("")

Now the prediction is correct. We have seen that something as simple as inverting the grayscale in an image can have a massive impact on the performance of a neural network.

**Now let us see what happens when we use an image of a hand-written 11. This time, the size, pixel values, and grayscale are the same as in the train dataset.**

In [ ]:
# Load image, convert it to grayscale, scale it and inverse it
!wget "https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Images/Handwritten%2011.jpg?raw=true"
image = plt.imread("/content/Handwritten 11.jpg?raw=true")
image = tf.image.rgb_to_grayscale(image)
image = image / 255
image = 1 - image
plt.imshow(image, cmap=plt.cm.gray)
plt.colorbar();

In [ ]:
# Predict class of image with model_4 and plot probabilities
prediction_image = model_4.predict(tf.expand_dims(image, axis=0), verbose=0)
fig, axes = plt.subplots(1,2, layout="constrained")
for i in range(2):
  axes[i].bar(height=prediction_image[0], x=list(range(10)))
  axes[i].set_ylabel("Probability")
  axes[i].set_xticks(list(range(10)))
  if i == 1:
    axes[i].set_yscale("log")
    axes[i].set_ylabel("")

Of course, our model cannot predict a handwritten 11 because it has never seen a number greater than 9. It also has no output probability for 11.

In general, ANNs can only perform well on data that has the same structure as the data on which the network was trained. However, the model will produce an output on differently structured data, often with high confidence.

Therefore, it cannot be underestimated how important it is to examine your data when using ANNs and to remain critical of the output.

Now that we know how basic image classification works, in **[Chapter 3 - Convolutional Neural Networks](https://github.com/ManuelGehl/IMPRS-Introduction-to-Neural-Networks-2023/blob/main/Chapter_3_Convolutional_Neural_Network.ipynb)**, we will look at how we can use the structure of images for our neural network.